# ADAML, machine learning: workshop 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import re
import re
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\niemelav\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Initialize NLTK resources
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# 0. Data importing and preprocessing

In [3]:
df = pd.read_csv("df_file.csv")

print(df.head())

print(df.isnull().sum()) # checking whether data contains any nulls

                                                Text  Label
0  Budget to set scene for election\n \n Gordon B...      0
1  Army chiefs in regiments decision\n \n Militar...      0
2  Howard denies split over ID cards\n \n Michael...      0
3  Observers to monitor UK election\n \n Minister...      0
4  Kilroy names election seat target\n \n Ex-chat...      0
Text     0
Label    0
dtype: int64


No nulls!

Next: shuffling the data, since the classes are ordered in the original data:

In [4]:
df = df.sample(frac=1).reset_index(drop=True) 
df.head()

,Text,Label
0,Deutsche Boerse boosts dividend\n \n Deutsche ...,4
1,Sayeed to stand down as Tory MP\n \n Tory MP J...,0
2,"'Best person' for top legal job\n \n The ""best...",0
3,US in EU tariff chaos trade row\n \n The US ha...,4
4,Top stars join US tsunami TV show\n \n Brad Pi...,3


Next: checking duplicates and dropping them if existing:

In [5]:
print(df.duplicated().sum())
df = df.drop_duplicates(ignore_index=True)
print(df.duplicated().sum())

print(df.isnull().sum())

98
0
Text     0
Label    0
dtype: int64


98 dublicates dropped

Next: defining a function which preprocesses the text

In [6]:
def clean_text(text):
    text = text.lower() # lowering the text
    text = re.sub(r'[^a-z\s]', '', text) # removes numbers, punctuations, uppercase letters and special symbols
    text = re.sub(r'\s+', ' ', text).strip() # collapses multiple spaces/tabs/newlines into one space and removes any spaces at the start or end
    words = text.split() # splitting text into words

    words = [lemmatizer.lemmatize(word) for word in words] # lemmatization

    words = [word for word in words if word not in stop_words] # stop-word removal

    return " ".join(words) # joining words and adding space between the words

In [7]:
df['new_text'] = df['Text'].apply(clean_text)

print("\nExample of cleaned text:")
print(df['new_text'].iloc[0])


Example of cleaned text:
deutsche boerse boost dividend deutsche boerse german stock exchange trying buy london rival ha said boost dividend payment analyst said move aimed winning investor opposed bid london stock exchange critic takeover complained money could better used returning cash shareholder deutsche boerse also said profit three month december wa euro sale climbed euro lifting revenue year record bn euro frankfurtbased deutsche boerse ha offered bn bn bn euro london stock exchange rival paneuropean bourse euronext working also bid late monday deutsche boerse said would lift dividend payment euro cent euro cent year earlier whiff sweetener anais faraj analyst nomura told bbcs world business report disgruntled shareholder deutsche boerse complaining money used bid could better placed hand paid dividend mr faraj continued deutsche boerse trying buy sense said


# 1. Data splitting

In [8]:
X = df["new_text"]
y = df["Label"]

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3) # test size = 20% of the data

# 3. Vectorization

Mapping words into integer IDs based on the frequency and vectorizing text to SEQUENCY_LENGTH length vectors of words integer IDs.

In [9]:
# Hyperparameters for Keras Vectorization
MAX_TOKENS = 65553 # = how many unique words to keep
SEQUENCE_LENGTH = 150 # longer texts are truncated and shorter texts are padded

# Initialize TextVectorization layer
vectorizer = layers.TextVectorization(
    max_tokens=MAX_TOKENS,
    output_mode="int", # outputs integer indices for words
    output_sequence_length=SEQUENCE_LENGTH
)

# Adapt the vectorizer to the training data
vectorizer.adapt(X_train) # building a mapping from words to integer IDs based on frequency

# Convert text data to sequences of integers
X_train_seq = vectorizer(X_train)
X_test_seq = vectorizer(X_test)

print(f"Shape of Keras sequences (Training): {X_train_seq.shape}")

Shape of Keras sequences (Training): (1701, 150)


In [10]:
X_train_seq

<tf.Tensor: shape=(1701, 150), dtype=int64, numpy=
array([[ 2893,  1893,   517, ...,     0,     0,     0],
       [ 9487,    57,  7931, ...,     0,     0,     0],
       [ 4068,  9801,   218, ...,  8002,  2017,     2],
       ...,
       [  377,    18,   297, ...,     0,     0,     0],
       [  292,   114, 15747, ...,   765,    54,   288],
       [  703,  4216,   348, ...,   575,    96,  1908]], shape=(1701, 150))>

# 4. Embedding and model training

Mapping each token ID (= word ID) to a dense vector of size 128. The model used is vanilla RNN (= each hidden state is calculated with tanh-function) with 64 hidden units. The activation function for the output layer used is softmax. For the evaluation accuracy score is used (= how many correct labels).

In [11]:
embedding_dim = 130  # each word will be presented as a 128-dimensional vector
num_classes = 5      # number of target classes

model = keras.Sequential([
    # Input: integer token sequences from your vectorizer
    layers.Embedding(
        input_dim=MAX_TOKENS, 
        output_dim=embedding_dim, 
        mask_zero=True,           # mask padding zeros
        input_length=SEQUENCE_LENGTH
    ),

    # 🔹 Simple Recurrent Layer (Vanilla RNN)
    layers.SimpleRNN(64, return_sequences=False), # outputs only the final hidden state

    # Output layer — softmax for 5 classes
    layers.Dense(num_classes, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",  # integer labels
    optimizer="adam", # adaptive learning rate optimizer
    metrics=["accuracy"]
)

EPOCHS = 10
BATCH_SIZE = 25

history = model.fit(
    X_train_seq, y_train,
    validation_data=(X_test_seq, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

Epoch 1/10


C:\Users\niemelav\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


69/69 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.4015 - loss: 1.3912 - val_accuracy: 0.4718 - val_loss: 1.2059
Epoch 2/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.7513 - loss: 0.8895 - val_accuracy: 0.5070 - val_loss: 1.0803
Epoch 3/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.9765 - loss: 0.3025 - val_accuracy: 0.4977 - val_loss: 1.1381
Epoch 4/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - accuracy: 1.0000 - loss: 0.0521 - val_accuracy: 0.4812 - val_loss: 1.1626
Epoch 5/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 1.0000 - loss: 0.0186 - val_accuracy: 0.4789 - val_loss: 1.2052
Epoch 6/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 1.0000 - loss: 0.0109 - val_accuracy: 0.4648 - val_loss: 1.2492
Epoch 7/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 1.0000 - loss: 0.0074 - val_accuracy: 0.4624 - val_loss: 1.2817
Epoch 8/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 1.0000 - loss: 0.0055 - val_accuracy: 0.4624 - val_loss: 1.

# 6. Testing

In [12]:
print("\n--- RNN Model Final Evaluation ---")
loss, acc = model.evaluate(X_test_seq, y_test, verbose=1)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {acc:.4f}")


--- RNN Model Final Evaluation ---
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4484 - loss: 1.3553
Loss: 1.3553
Accuracy: 0.4484


Pretty bad accuracy!